In [ ]:
import os.path
import numpy

import pygetm
import pygetm.legacy

# Note: "%matplotlib widget" below enables interactive plots but requires https://github.com/matplotlib/ipympl
# Alternatively you could use "%matplotlib notebook" (deprecated).
%matplotlib widget
import matplotlib.pyplot

In [ ]:
# Load North Sea domain
getm_cases = next(filter(os.path.isdir, ('/data/kb/getm-setups', '../../../getm-setups')))
domain = pygetm.legacy.domain_from_topo(os.path.join(getm_cases, 'NorthSea/Topo/NS6nm.v01.nc'), nlev=30, z0_const=0.001)
sim = pygetm.Simulation(domain, runtype=2, fabm='../../extern/fabm/testcases/fabm-jrc-med_ergom.yaml')
sim.fabm_model.fabm.configure(1)

nc = sim.output_manager.add_netcdf_file('bgc.nc', interval=10)
nc.request('med_ergom_pp')

In [ ]:
sim.fabm_model.interior_dependencies['downwelling_photosynthetic_radiative_flux'].value = 5
sim.fabm_model.interior_dependencies['temperature'].value = 15
sim.fabm_model.interior_dependencies['practical_salinity'].value = 30
sim.fabm_model.horizontal_dependencies['surface_downwelling_photosynthetic_radiative_flux'].value = 50
sim.fabm_model.horizontal_dependencies['wind_speed'].value = 50
sim.fabm_model.horizontal_dependencies['bottom_stress'].value = 0

sim.start_fabm()

In [ ]:
# Simulate for 1 day
dt = 1800
for _ in range(int(86400 / dt)):
    sources_int, sources_sf, sources_bt = sim.fabm_model.get_sources()
    #print(sources_int.min(), sources_int.max())
    sim.fabm_model.interior_state += sources_int * dt
    sim.fabm_model.surface_state += sources_sf * dt
    sim.fabm_model.bottom_state += sources_bt * dt
    sim.output_manager.save()
sim.output_manager.close()

In [ ]:
# Plot surface field and y-z slice 
fig, (ax1, ax2) = matplotlib.pyplot.subplots(ncols=2, figsize=(12, 4))
pc = ax1.pcolormesh(sim.fabm_model.interior_state[8, -1, ...])
pc = ax2.pcolormesh(sim.fabm_model.interior_state[8, :, :, 55])
fig.colorbar(pc);